# Rossmann Store Sales Forecasting Guide with XGBoost

This notebook walks through a full, end-to-end workflow for forecasting Rossmann store sales using the gradient boosting library XGBoost. Follow along, execute the cells sequentially, and adapt each section for deeper experimentation.

## How to Use this Guide
- **Set the stage**: import core analytics libraries and load the Rossmann CSV files.
- **Explore and diagnose**: compute summary stats, investigate missing values, and inspect temporal patterns.
- **Engineer signal**: enrich the raw timeline with calendar features, competition indicators, and promo flags.
- **Model and tune**: assemble a preprocessing + XGBoost pipeline, run randomized hyperparameter tuning, and judge performance.
- **Wrap up**: refit the best model on all data and generate test-set forecasts in the competition format.

> ⚙️ **Prerequisites**: Ensure `xgboost` is available in your environment (`pip install xgboost`). All other libraries ship with typical scientific Python distributions.

In [1]:
# ============================================================
# 1. Import common scientific Python libraries and utilities.
#    Adjust this block if you prefer alternatives (e.g., plotly).
# ============================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from IPython.display import display

pd.options.display.max_columns = 120
sns.set(style='whitegrid', context='talk')
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
plt.rcParams['figure.figsize'] = (12, 6)


## Step 1 · Load the Rossmann data files
We will work with the Kaggle Rossmann challenge files already present in `../data/rossmann/`. The training data contains historical sales, while `store.csv` adds meta-information such as store type and competition details.

In [ ]:
# ============================================================
# 2. Load base CSVs into pandas DataFrames and confirm shapes.
#    Using Path keeps the code robust if folders move around.
# ============================================================
data_dir = Path('../data/rossmann')
train_path = data_dir / 'train.csv'
test_path = data_dir / 'test.csv'
store_path = data_dir / 'store.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
store_df = pd.read_csv(store_path)

print(f'Train shape: {train_df.shape}')
print(f'Test shape:  {test_df.shape}')
print(f'Store shape: {store_df.shape}')


## Step 2 · Quick first look
Use rich table displays to sanity-check the import and familiarize yourself with available columns.

In [ ]:
# ============================================================
# 3. Peek at the head of each dataframe to inspect fields.
#    `display` keeps notebooks readable compared to raw print.
# ============================================================
display(train_df.head())
display(store_df.head())
display(test_df.head())


## Step 3 · Baseline diagnostics
We want to check column data types and identify potential data quality issues (nulls, string representations of categories, etc.).

In [ ]:
# ============================================================
# 4. Summarize schema information: dtypes + non-null counts.
#    `.info()` prints directly; no need to wrap in display().
# ============================================================
train_df.info()
print('
' + '-' * 80 + '
')
store_df.info()


## Step 4 · Explore missing values
A quick percentage table points us toward the columns that need special handling before modeling.

In [ ]:
# ============================================================
# 5. Compute missing-value ratios to guide data cleaning.
#    Sorting helps surface the most problematic columns first.
# ============================================================
def missing_report(df, name):
    pct_missing = df.isna().mean().mul(100).sort_values(ascending=False)
    summary = pct_missing[pct_missing > 0].to_frame(name).round(2)
    if summary.empty:
        print(f'No missing data detected in {name}.')
    else:
        display(summary)

missing_report(train_df, 'train_missing_%')
missing_report(store_df, 'store_missing_%')


## Step 5 · Understand the target (Sales)
Sales is right-skewed; inspecting histograms in both raw and log space usually informs loss functions and transforms.

In [ ]:
# ============================================================
# 6. Visualize sales distribution (raw vs. log1p) to evaluate
#    the extent of skewness and potential benefits of log-scaling.
# ============================================================
fig, axes = plt.subplots(1, 2, figsize=(18, 6))
sns.histplot(train_df['Sales'], bins=50, ax=axes[0], color='steelblue')
axes[0].set_title('Sales Distribution (Raw)')
axes[0].set_xlabel('Sales')

sns.histplot(np.log1p(train_df['Sales']), bins=50, ax=axes[1], color='darkorange')
axes[1].set_title('Sales Distribution (log1p transformed)')
axes[1].set_xlabel('log1p(Sales)')
plt.tight_layout()
plt.show()


## Step 6 · Time-series intuition
Aggregating total sales over time reveals macro trends, seasonality, and anomalies (e.g., holidays).

In [ ]:
# ============================================================
# 7. Plot aggregated daily sales to spot level shifts or
#    promotions affecting all stores. Grouping smooths noise.
# ============================================================
train_df['Date'] = pd.to_datetime(train_df['Date'])
daily_sales = train_df.groupby('Date')['Sales'].sum()
daily_sales.plot(title='Aggregate Daily Sales Across Stores')
plt.ylabel('Total Sales')
plt.xlabel('Date')
plt.show()


## Step 7 · Merge store metadata
Before heavy feature engineering, join `store.csv` so that each record carries competition, assortment, and promo information.

In [ ]:
# ============================================================
# 8. Merge store-level attributes into train/test copies
#    for richer exploratory plots and feature creation.
# ============================================================
train_merged = train_df.merge(store_df, on='Store', how='left')
test_merged = test_df.merge(store_df, on='Store', how='left')

print(train_merged.shape, test_merged.shape)
display(train_merged.head())


## Step 8 · Feature engineering helper
To keep transformations consistent between train and test sets, we will build a single function that applies all cleaning and enrichment steps. The function copies inputs to avoid modifying the original DataFrames.

In [ ]:
# ============================================================
# 9. Define a reusable preparation function. Each step is
#    commented so you can tweak or extend the feature logic.
# ============================================================
def prepare_rossmann(raw_df: pd.DataFrame, store_meta: pd.DataFrame, *, is_train: bool) -> pd.DataFrame:
    # Work on a copy to avoid side effects when experimenting.
    df = raw_df.copy()

    # Merge store metadata and parse dates early for reuse.
    df = df.merge(store_meta, on='Store', how='left')
    df['Date'] = pd.to_datetime(df['Date'])

    # Canonicalize categorical codes to descriptive labels.
    state_map = {'0': 'None', 'a': 'PublicHoliday', 'b': 'EasterHoliday', 'c': 'Christmas'}
    df['StateHoliday'] = df['StateHoliday'].astype(str).replace(state_map)

    # Fill straightforward numeric gaps with sensible defaults.
    df['Open'] = df['Open'].fillna(1).astype(int)
    df['Promo'] = df['Promo'].fillna(0).astype(int)
    df['SchoolHoliday'] = df['SchoolHoliday'].fillna(0).astype(int)
    df['Promo2'] = df['Promo2'].fillna(0).astype(int)
    df['CompetitionDistance'] = df['CompetitionDistance'].fillna(df['CompetitionDistance'].median())

    # When competition/promo start dates are missing, assume
    # they became active at the current record's date.
    df['CompetitionOpenSinceYear'] = df['CompetitionOpenSinceYear'].fillna(df['Date'].dt.year)
    df['CompetitionOpenSinceMonth'] = df['CompetitionOpenSinceMonth'].fillna(df['Date'].dt.month)
    df['Promo2SinceYear'] = df['Promo2SinceYear'].fillna(df['Date'].dt.year)
    df['Promo2SinceWeek'] = df['Promo2SinceWeek'].fillna(df['Date'].dt.isocalendar().week.astype(int))

    # Calendar decompositions unlock seasonal signal for the model.
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['WeekOfYear'] = df['Date'].dt.isocalendar().week.astype(int)
    df['DayOfYear'] = df['Date'].dt.dayofyear
    df['IsWeekend'] = df['DayOfWeek'].isin([6, 7]).astype(int)

    # Promo interval strings signal recurring campaigns.
    df['PromoInterval'] = df['PromoInterval'].fillna('None')
    df['MonthStr'] = df['Date'].dt.strftime('%b')
    df['IsPromoMonth'] = df.apply(
        lambda row: int(row['PromoInterval'] != 'None' and row['MonthStr'] in row['PromoInterval'].split(',')), axis=1
    )

    # Competition and promo recency features capture duration effects.
    df['CompetitionMonthsOpen'] = (
        (df['Year'] - df['CompetitionOpenSinceYear']) * 12
        + (df['Month'] - df['CompetitionOpenSinceMonth'])
    ).clip(lower=0)
    df['Promo2WeeksActive'] = (
        (df['Year'] - df['Promo2SinceYear']) * 52
        + (df['WeekOfYear'] - df['Promo2SinceWeek'])
    ).clip(lower=0)

    # Stabilize distance by log-scaling; large ranges can slow learning.
    df['CompetitionDistanceLog'] = np.log1p(df['CompetitionDistance'])

    # Guard against potential negative or null durations.
    df['CompetitionMonthsOpen'] = df['CompetitionMonthsOpen'].fillna(0)
    df['Promo2WeeksActive'] = df['Promo2WeeksActive'].fillna(0)

    # Drop columns not available at prediction time or redundant after engineering.
    df = df.drop(columns=['Customers'], errors='ignore')
    df = df.drop(columns=['MonthStr'], errors='ignore')

    # Create a log-transformed sales column for regression stability.
    if is_train:
        df = df[df['Sales'] > 0]  # Closed days already have zero sales.
        df['Sales_Log'] = np.log1p(df['Sales'])
    else:
        df['Sales'] = np.nan
        df['Sales_Log'] = np.nan

    return df


## Step 9 · Apply the feature pipeline
Generate processed training and test tables. A quick shape + head check confirms that the function behaved as expected.

In [ ]:
# ============================================================
# 10. Run the preparation helper on train/test sets.
#     Keep the original frames intact for reference/EDA.
# ============================================================
train_ready = prepare_rossmann(train_df, store_df, is_train=True)
test_ready = prepare_rossmann(test_df, store_df, is_train=False)

print(train_ready.shape, test_ready.shape)
display(train_ready.head())


## Step 10 · Create a time-aware validation split
Rossmann data is temporal, so we hold out the most recent six weeks as a validation set. This better mimics real forecasting than a random split.

In [ ]:
# ============================================================
# 11. Split chronologically: everything before the cutoff is
#     training data, the tail segment becomes validation.
# ============================================================
train_ready = train_ready.sort_values('Date')
split_date = train_ready['Date'].max() - pd.Timedelta(weeks=6)
train_mask = train_ready['Date'] < split_date

feature_drop = ['Sales', 'Sales_Log', 'Date']
X_train = train_ready.loc[train_mask].drop(columns=feature_drop + ['Id'], errors='ignore')
y_train = train_ready.loc[train_mask, 'Sales_Log']
X_val = train_ready.loc[~train_mask].drop(columns=feature_drop + ['Id'], errors='ignore')
y_val = train_ready.loc[~train_mask, 'Sales_Log']

print(f'Training rows: {X_train.shape[0]}')
print(f'Validation rows: {X_val.shape[0]}')


## Step 11 · Build preprocessing + model pipeline
Leverage `ColumnTransformer` to one-hot encode categorical variables while scaling numeric ones. The pipeline keeps feature handling consistent during cross-validation.

In [ ]:
# ============================================================
# 12. Identify categorical vs numeric columns dynamically so
#     the pipeline adapts if you add/remove engineered fields.
# ============================================================
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()
numeric_cols = X_train.columns.difference(categorical_cols).tolist()

print('Categorical columns:', categorical_cols)
print('Numeric columns:', numeric_cols[:10], '...')

preprocess = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols),
    ]
)

xgb_reg = XGBRegressor(
    objective='reg:squarederror',
    eval_metric='rmse',
    tree_method='hist',
    random_state=RANDOM_STATE,
    n_estimators=400,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
)

pipeline = Pipeline(
    steps=[('preprocess', preprocess), ('model', xgb_reg)]
)


## Step 12 · Hyperparameter tuning
We run a randomized search over a compact grid. Feel free to widen the ranges or increase `n_iter` once the workflow is solid.

In [ ]:
# ============================================================
# 13. Set up randomized search for XGBoost hyperparameters.
#     RMSE on the log-scaled target is the optimization metric.
# ============================================================
param_distributions = {
    'model__n_estimators': [200, 400, 600, 800],
    'model__max_depth': [3, 4, 5, 6, 8],
    'model__learning_rate': [0.03, 0.05, 0.1, 0.2],
    'model__subsample': [0.6, 0.8, 1.0],
    'model__colsample_bytree': [0.6, 0.8, 1.0],
    'model__min_child_weight': [1, 3, 5, 7],
    'model__gamma': [0, 0.05, 0.1, 0.3],
}

random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=20,
    scoring='neg_root_mean_squared_error',
    cv=3,
    verbose=2,
    n_jobs=-1,
    random_state=RANDOM_STATE,
)

random_search.fit(X_train, y_train)

print('Best RMSE (log space):', -random_search.best_score_)
print('Best params:')
for k, v in random_search.best_params_.items():
    print(f'  {k}: {v}')


## Step 13 · Evaluate on validation set
Convert predictions back to the natural scale with `expm1` and compute RMSE / MAE for interpretability.

In [ ]:
# ============================================================
# 14. Score the tuned pipeline on the hold-out validation slice
#     to confirm the search produced a stronger configuration.
# ============================================================
best_pipeline = random_search.best_estimator_
val_pred_log = best_pipeline.predict(X_val)
val_pred = np.expm1(val_pred_log)
val_actual = np.expm1(y_val)

val_rmse = mean_squared_error(val_actual, val_pred, squared=False)
val_mae = mean_absolute_error(val_actual, val_pred)
print(f'Validation RMSE: {val_rmse:,.2f}')
print(f'Validation MAE:  {val_mae:,.2f}')


## Step 14 · Inspect feature contributions
A quick feature importance review spotlights which engineered fields drive the model.

In [ ]:
# ============================================================
# 15. Combine the preprocessor's feature names with XGBoost's
#     gain-based importances to see top drivers.
# ============================================================
feature_names = best_pipeline.named_steps['preprocess'].get_feature_names_out()
importances = best_pipeline.named_steps['model'].feature_importances_
importance_df = (
    pd.DataFrame({'feature': feature_names, 'importance': importances})
    .sort_values('importance', ascending=False)
    .head(20)
)
display(importance_df)


## Step 15 · Refit on all available data
Once validation looks good, retrain the tuned pipeline on the full training span (train + validation) to maximize signal before forecasting the test set.

In [ ]:
# ============================================================
# 16. Refit the best pipeline on the complete training window.
#     This leverages every historical observation for final preds.
# ============================================================
X_full = train_ready.drop(columns=['Sales', 'Sales_Log', 'Date', 'Id'], errors='ignore')
y_full = train_ready['Sales_Log']
best_pipeline.fit(X_full, y_full)


## Step 16 · Generate competition-style predictions
Apply the final pipeline to the prepared test set, invert the log-transform, and package predictions with the `Id` column.

In [ ]:
# ============================================================
# 17. Score the test rows and create a submission-ready frame.
# ============================================================
test_features = test_ready.drop(columns=['Sales', 'Sales_Log', 'Date'], errors='ignore')
test_ids = test_ready['Id'] if 'Id' in test_ready.columns else test_df['Id']
test_pred_log = best_pipeline.predict(test_features)
test_predictions = np.expm1(test_pred_log)
submission = pd.DataFrame({'Id': test_ids, 'Sales': test_predictions})
display(submission.head())


## Step 17 · Save for Kaggle submission (optional)
Uncomment the final line to persist a CSV that you can submit to Kaggle for the Rossmann challenge.

In [ ]:
# ============================================================
# 18. Write predictions to disk if you want to submit results.
# ============================================================
# output_path = Path('rossmann_xgboost_submission.csv')
# submission.to_csv(output_path, index=False)
# print(f'Submission file saved to {output_path.resolve()}')


## Where to go next
- Compare `RandomizedSearchCV` with time-series aware CV (e.g., `TimeSeriesSplit`).
- Engineer store-level aggregates (rolling means, promotion cadence).
- Blend multiple models (e.g., LightGBM, CatBoost) for ensemble gains.
- Track experiments with MLflow or Weights & Biases for reproducibility.